# Desarrollo de un sistema de diagnóstico de enfermedades en hojas de tomate mediante PlantCV y modelos de aprendizaje profundo

La detección temprana de enfermedades en cultivos es vital para la agricultura sostenible. Utilizando PlantCV (https://plantcv.readthedocs.io/), una biblioteca enfocada en la visión por computadora para plantas, junto con TensorFlow o PyTorch, el alumno se encargará de crear modelos de clasificación multiclase robustos. Estos modelos tendrán como objetivo identificar 10 distintas enfermedades y una clase de salud en hojas de tomate. Para el entrenamiento y evaluación, se usará el Tomato Leaves Dataset de Kaggle (https://www.kaggle.com/datasets/ashishmotwani/tomato), que contiene más de 20,000 imágenes recogidas en entornos de laboratorio e in situ. Dada la naturaleza práctica de la aplicación, se buscará desarrollar un modelo lo suficientemente eficiente como para poder ser implementado en aplicaciones móviles, permitiendo diagnósticos ágiles y efectivos en el campo.

## Carga de la data

In [ ]:
# codigo para cargar la data y prepararla

## Exploracion de los datos

In [ ]:
# codigo para explorar caracteristicas escenciales, con plantCV y demas librerias
# para ver tamanos, balance de train y validacion, separar un poco para el test

## Modelos de clasificacion multiclases

In [ ]:
# Se crean los modelos y se los entrenan

## Comparacion de los mejores modelos obtenidos

In [ ]:
# Se compara los modelos obtenidos y se decide por el mejor

## Predicciones con el mejor modelo para este caso

In [ ]:
#Se crea una clase para hacer la prediccion en base a una imagen nueva